### 

In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
import utils


In [41]:
events = utils.get_events()
events.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,month,day,hora
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0


In [42]:
df_viewed_checkout = events[(events['event'] == 'viewed product') | ((events['event'] == 'checkout'))]
df_viewed_checkout = df_viewed_checkout[['timestamp', 'person', 'event', 'sku']]
df_viewed_checkout.head()

,timestamp,person,event,sku
0,2018-05-18 00:11:59,4886f805,viewed product,9288.0
1,2018-05-18 00:11:27,ad93850f,viewed product,304.0
2,2018-05-18 00:11:16,0297fc1e,viewed product,6888.0
3,2018-05-18 00:11:14,2d681dd8,viewed product,11890.0
4,2018-05-18 00:11:09,cccea85e,viewed product,7517.0


In [43]:
df_viewed_checkout = df_viewed_checkout.groupby(['person', 'event', 'sku']).agg({'timestamp':'count'})
df_viewed_checkout = df_viewed_checkout.reset_index()
df_viewed_checkout.head()

,person,event,sku,timestamp
0,0008ed71,checkout,3372.0,1
1,0008ed71,checkout,7505.0,1
2,0008ed71,checkout,8247.0,1
3,00091926,checkout,6901.0,1
4,00091926,checkout,6930.0,1


In [44]:
df_viewed_checkout = df_viewed_checkout.pivot_table(index=['person','sku'], columns='event', values='timestamp', aggfunc='count', fill_value=0)
df_viewed_checkout.head()

event            checkout  viewed product
person   sku                             
0008ed71 3372.0         1               0
         7505.0         1               0
         8247.0         1               0
00091926 1365.0         0               1
         1373.0         0               1

In [45]:
index_df = df_viewed_checkout.index.to_frame(index=False)
df_viewed_checkout = df_viewed_checkout.reset_index(drop=True)
df_viewed_checkout = pd.merge(index_df, df_viewed_checkout, left_index=True, right_index=True)
df_viewed_checkout.head()

,person,sku,checkout,viewed product
0,0008ed71,3372.0,1,0
1,0008ed71,7505.0,1,0
2,0008ed71,8247.0,1,0
3,00091926,1365.0,0,1
4,00091926,1373.0,0,1


In [60]:
df_viewed_checkout['viewed/checkouted'] = df_viewed_checkout['checkout']/df_viewed_checkout['viewed product']
df_viewed_checkout['viewed/checkouted'] = df_viewed_checkout['viewed/checkouted'].replace(np.inf, -1)
df_viewed_checkout.head()

,person,sku,checkout,viewed product,viewed/checkouted
0,0008ed71,3372.0,1,0,-1.0
1,0008ed71,7505.0,1,0,-1.0
2,0008ed71,8247.0,1,0,-1.0
3,00091926,1365.0,0,1,0.0
4,00091926,1373.0,0,1,0.0


In [62]:
df_viewed_checkout = df_viewed_checkout.groupby('person').agg({'viewed/checkouted':('max','min','mean','sum')})
df_viewed_checkout

viewed/checkouted                    
                       max  min      mean  sum
person                                        
0008ed71              -1.0 -1.0 -1.000000 -3.0
00091926               1.0  0.0  0.012195  2.0
00091a7a               0.0  0.0  0.000000  0.0
000ba417               1.0  0.0  0.049383  4.0
000c79fe               1.0  1.0  1.000000  1.0
000e4d9e               1.0  0.0  0.009524  1.0
000e619d               1.0  0.0  0.040000  1.0
001001be               1.0  0.0  0.052632  1.0
0010e89a               1.0  1.0  1.000000  1.0
0016c4b5               1.0  1.0  1.000000  1.0
001802e4               1.0  0.0  0.500000  1.0
001804a2              -1.0 -1.0 -1.000000 -2.0
0019e639               1.0  0.0  0.074627  5.0
001a2273               1.0  0.0  0.500000  1.0
001b0bf9               1.0  1.0  1.000000  2.0
001ca5ee               1.0  0.0  0.032258  1.0
001dfc31               0.0  0.0  0.000000  0.0
001e9aea               1.0  0.0  0.272727  3.0
0020152e               1.0  0.0  0.500000  1.0
00202cdb               1.0  0.0  0.166667  1.0
0022965d               1.0  0.0  0.444444  4.0
0027574e               1.0  0.0  0.030303  2.0
00295d74               1.0  0.0  0.166667  1.0
002aea56               1.0  0.0  0.079545  7.0
002b0188               1.0  0.0  0.090909  1.0
002e74b2               1.0  0.0  0.500000  1.0
002ed810               1.0  0.0  0.111111  1.0
002fb981               1.0  0.0  0.076923  1.0
00317e49               1.0  0.0  0.250000  1.0
0031c75f               1.0  0.0  0.100000  1.0
...                    ...  ...       ...  ...
ffc670b3               1.0  0.0  0.071429  4.0
ffcaae19               1.0  0.0  0.062500  1.0
ffcaeaae               0.0  0.0  0.000000  0.0
ffccd5a1               1.0  0.0  0.400000  2.0
ffcf952b               1.0  0.0  0.025641  1.0
ffd41214               0.0  0.0  0.000000  0.0
ffd62616               1.0  0.0  0.125000  1.0
ffd9756a               1.0  1.0  1.000000  1.0
ffda14ca               1.0  1.0  1.000000  1.0
ffdafc72               1.0  0.0  0.250000  1.0
ffddd0ec               0.0  0.0  0.000000  0.0
ffde3628               1.0  0.0  0.039216  2.0
ffe066cc               1.0  0.0  0.076923  1.0
ffe53446               1.0 -1.0  0.021739  1.0
ffe7e848               1.0  0.0  0.105263  4.0
ffeb80c5               1.0  1.0  1.000000  1.0
ffebdbc9               1.0  0.0  0.142857  1.0
ffecdf29               1.0  0.0  0.333333  1.0
ffecea3d               0.0  0.0  0.000000  0.0
ffed0342               1.0  0.0  0.032609  3.0
ffed3d0e               1.0  1.0  1.000000  1.0
ffee0f18               1.0  0.0  0.003170  2.0
ffef83e6               1.0  0.0  0.014493  1.0
fff1659c               1.0  0.0  0.071429  1.0
fff1b11a               0.0  0.0  0.000000  0.0
fff1caee               1.0  1.0  1.000000  1.0
fff2bdde               1.0  0.0  0.136364  3.0
fff72025               1.0  0.0  0.045455  1.0
fff78145               1.0  0.0  0.062500  1.0
fffd1246               1.0  0.0  0.333333  1.0

[38241 rows x 4 columns]

In [ ]:
#df_viewed.to_csv('features_productos_checkouts.csv',index=False)